## LeNet-5 数据集MNIST
![](LeNet-5_layers.png) 
注：<br>
图片中的层次结构是没有经过padding的，所以在第一次全连阶时神经元个数为16\*4\*4。<br>
若在第一次卷积时，经过2-padding，则是16\*5\*5

### 导入神经网络需要的包

In [ ]:
import torch
import torch.nn as nn             # 专门为神经网络设计的模块化接口
import torch.nn.functional as F   # 包含很多CNN需要的函数，如激活函数、dropout函数
import torch.optim as optim       # 包含CNN需要的优化器，用于后向传播更新参数
from torchvision import datasets, transforms  # 加载数据集，对数据集进行处理

### 设定超级参数

In [ ]:
BATCH_SIZE = 512 #一次训练所抓取的数据样本数量
EPOCHS = 20 # 总训练次数

# 让torch判断是否使用GPU（更快） 但是目前Mac还不支持GPU，可以放到Colab上
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

### 载入数据
#### 训练集-batch_size x 1 x 28 x28
一共有batch_size张图片，图片是单通道（黑白），长宽为28

In [ ]:
train_loader = torch.utils.data.DataLoader(  # 这个数据可以也载入自定义数据
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(), # 把数据变为tensor的形式
                           
                           # 归一化：data减去它的均值，再除以它的标准差。这两个值是官方给定好的值
                           transforms.Normalize((0.1307,), (0.3081,)) 
                       ])), 
        batch_size=BATCH_SIZE, shuffle=True)  # shuffle随机打乱操作

# 这是一个数据加载器，用来把训练数据分成多个小组，此函数每次抛出一组数据，数据大小为batch_size
# 每组数据都会进行一次梯度下降，对于整个数据集来说，一次训练，总共进行了(total/batch_size)次

#### 测试集

In [ ]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

### 构建网络

In [ ]:
class LeNet5(nn.Module):

    def __init__(self):
        super(LeNet5, self).__init__()
        
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)  # 1 input image channel, 6 output channels, 5x5 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5) # 输入通道6， 输出通道16， 卷积核5x5
        
        # an affine operation: y = Wx + b
        # 【本来这里论文中是最后一个卷积层，但是官网代码写的是线性层】
        self.fc1 = nn.Linear(16*5*5, 120) # 神经元个数计算 = kernel的个数 * 图片现存大小
        
        # 【真正意义上】全连接层：对特征进行计算
        self.fc2 = nn.Linear(120, 84)  # 全连接函数为线性函数：y = Wx + b，并将120个节点连接到84个节点上。
        self.fc3 = nn.Linear(84, 10)   # “84” 好像是什么图像的大小？？

    def forward(self, x): #（卷积1-激活）-池化-（卷积2-激活）-池化-整合为一维-（全连阶1-激活）-（全连阶2-激活）-全连阶3
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x)) #输入x经过全连接1，再经过ReLU激活函数，然后更新x
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        # 计算log(softmax(x)) 每个输出分类的结果都赋予一个概率值【以“行”为单位进行处理】
#         x = F.log_softmax(x, dim=1) 
        return x

### 选择优化器

In [ ]:
model = LeNet5().to(DEVICE)
optimizer = optim.Adam(model.parameters())

# 设定损失函数
criterion = nn.CrossEntropyLoss() 
# CrossEntropyLoss是log_softmax+nll_loss的合并

### 构建训练函数

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) #把数据移到GPU上
        
        optimizer.zero_grad() # 清除之前的误差
        
        output = model(data)  # 训练数据，得到结果
        
#         loss = F.nll_loss(output, target) # 计算误差
        loss = criterion(output, target) 
        
        loss.backward()  #反向传播，更新参数
        optimizer.step()
        
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### 构建测试函数
model.train() 和 model.eval() 一般在模型训练和评价的时候会加上这两句，主要是针对由于model 在训练时和评价时 Batch Normalization 和 Dropout 方法模式不同；（暂时先记住）

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): # 暂时不追踪参数的梯度？？
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            output = model(data) # 给出预测
            
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### 主函数-训练与检测

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)